In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f
from dataclasses import dataclass


In [2]:
spark: SparkSession = SparkSession.builder.master("local[*]").appName("Spark SQL Example").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/04 11:01:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataset = spark.read.parquet("../../data/replicated-lead-variants.parquet")


In [4]:
dataset.repartition(100)


DataFrame[variantId: string, studyId: string, studyLocusId: string, beta: double, zScore: double, pValueMantissa: float, pValueExponent: int, standardError: double, finemappingMethod: string, studyType: string, credibleSetSize: int, isTransQtl: boolean, posteriorProbability: double, nSamples: int, nControls: int, nCases: int, geneId: string, traitFromSourceMappedIds: array<string>, majorPopulation: struct<ldPopulation:string,relativeSampleSize:double>, allelefrequencies: array<struct<populationName:string,alleleFrequency:double>>, vepEffect: struct<assessment:string,normalisedScore:double,targetId:string>, majorPopulationAF: array<struct<populationName:string,alleleFrequency:double>>, majorPopulationMAF: double, leadVariantStats: struct<chi2Stat:double,pValue:double,varianceExplained:double>, rescaledStatistics: struct<estimatedBeta:double,traitClass:string,chi2Stat:double,estimatedSE:double,varG:double,prev:double,nSamples:int>, MAFClass: string, vepScoreClass: string, estBetaClass: s

In [6]:
# Solution 1 -  Cross join the dateset to itself to create MAF overlapping bins
threshold = 0.01
left = dataset.select(f.col("variantId").alias("leftVariantId"), f.col("majorPopulationMAF").alias("leftMAF"))
df = dataset.join(
    left,
    how="inner",
    on=(
        (dataset.variantId != left.leftVariantId)
        & (dataset.majorPopulationMAF < (left.leftMAF + threshold))
        & (dataset.majorPopulationMAF >= (left.leftMAF - threshold))
    ),
).withColumn("MAF+/-{threshold}", f.mean("leftMAF").over(Window.partitionBy("majorPopulationMAF")))


df.write.parquet("../../data/replicated-lead-variants-maf-overlapping-bins.parquet", mode="overwrite")


25/06/04 11:05:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:KeyboardInterrupt while sending command.==>              (3 + 1) / 4]
Traceback (most recent call last):
  File "/Users/ss60/Projects/Gentropy-manuscript/.venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ss60/Projects/Gentropy-manuscript/.venv/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ss60/.local/share/uv/python/cpython-3.11.11-macos-aarch64-none/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# First binnarize the MAF dataset
breaks = list(range(0.0, 0.5, 0.01))


@dataclass
class Bin:
    lower: float
    upper: float


bins: list[Bin] = []
for idx, breakpoint in enumerate(breaks):
    if idx < len(breaks) - 1:
        break
    bins.append(Bin(lower=breakpoint, upper=breaks[idx + 1]))

# def binnarize_maf(maf: Column, bins: list)
